# Men's team unlikely to medal in floor

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('../../Combine_Data/men/fx_encoded.csv')
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_fx.csv')
fxnames = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_fxnames.csv')
fxolymp = pd.read_csv('../../Data/cleandata22-23/men22_23.csv')


In [3]:
df.head()

,Rank,D,E,ND,Total,year,medal,Name,Nation,round_final,round_qual
0,1,6.2,8.366,0.0,14.566,2019,0,62,55,True,False
1,2,5.9,8.600,0.0,14.500,2019,0,122,40,True,False
2,3,6.1,8.400,0.0,14.500,2019,1,262,25,True,False
3,3,6.1,8.400,0.0,14.500,2019,0,90,80,True,False
4,5,6.2,8.000,0.0,14.200,2019,0,168,63,True,False


In [4]:
y = df['medal']
X = df.drop(columns=['medal'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network": MLPClassifier(),
}

results = {}

# Iterate through each classifier
for name, clf in classifiers.items():
    # Create a pipeline with StandardScaler for classifiers that require it
    if name in ["SVM", "K-Nearest Neighbors", "Neural Network"]:
        clf = make_pipeline(StandardScaler(), clf)

    # Fit the model to the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    results[name] = {
        "accuracy": accuracy,
        "classification_report": report,
        "confusion_matrix": conf_matrix,
    }

# Print the results
for name, result in results.items():
    print(f"Classifier: {name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Classification Report:\n{result['classification_report']}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")


/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

Classifier: Random Forest
Accuracy: 0.9815
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       105
           1       1.00      0.33      0.50         3

    accuracy                           0.98       108
   macro avg       0.99      0.67      0.75       108
weighted avg       0.98      0.98      0.98       108

Confusion Matrix:
[[105   0]
 [  2   1]]

Classifier: AdaBoost
Accuracy: 0.9815
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       105
           1       1.00      0.33      0.50         3

    accuracy                           0.98       108
   macro avg       0.99      0.67      0.75       108
weighted avg       0.98      0.98      0.98       108

Confusion Matrix:
[[105   0]
 [  2   1]]

Classifier: SVM
Accuracy: 0.9722
Classification Report:
              precision    recall  f1-score   support

           0       0.97 

/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Classifier: Random Forest
Accuracy: 0.9815

Classifier: AdaBoost
Accuracy: 0.9815

In [7]:
X_train.head()

,Rank,D,E,ND,Total,year,Name,Nation,round_final,round_qual
386,49,5.3,8.400,0.0,13.700,2018,312,44,False,True
231,152,4.5,8.433,-0.4,12.533,2019,110,50,False,True
349,12,5.9,8.300,0.0,14.200,2018,186,27,False,True
448,111,4.8,8.500,-0.3,13.000,2018,31,8,False,True
302,9,5.4,8.500,0.0,27.800,2020,68,76,True,False


In [8]:
olymp.head()


,Rank,D Score,E Score,Pen.,Total,year,Name,nation,round_AAfinal,round_TeamFinal,round_TeamQual,round_final,round_qual
0,54.0,4.0,8.566,0.0,12.566,2023,236,67,False,False,False,False,True
1,13.0,5.5,8.300,0.0,13.800,2022,527,30,True,False,False,False,False
2,48.0,5.5,7.900,0.0,13.400,2022,527,30,False,False,False,False,True
3,86.0,5.4,7.133,0.1,12.433,2022,527,30,False,False,False,False,True
4,21.0,5.5,8.666,0.0,14.166,2023,527,30,False,False,False,False,True


In [37]:
olymp.rename(columns={'D Score': 'D', 'E Score': 'E',
                     'Pen.': 'ND', 'nation': 'Nation'}, inplace=True)

In [38]:
olymp=olymp.drop(columns=['round_TeamFinal', 'round_AAfinal', 'round_TeamQual']) 

In [39]:
y = df['medal']
X = df.drop(columns=['medal'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=42)

# Initialize the AdaBoost classifier
clf = AdaBoostClassifier(n_estimators=31)

# Fit the model to the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the results
print("AdaBoost Classifier")
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")

AdaBoost Classifier
Accuracy: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        62
           1       1.00      1.00      1.00         3

    accuracy                           1.00        65
   macro avg       1.00      1.00      1.00        65
weighted avg       1.00      1.00      1.00        65

Confusion Matrix:
[[62  0]
 [ 0  3]]


In [40]:
y_pred = clf.predict(olymp)


In [41]:
ypreds  = pd.Series(y_pred)
olymp['ypred']=ypreds


In [42]:
olymp['ypred'].value_counts()

ypred
0    2411
1      28
Name: count, dtype: int64

In [43]:
filtered_df = olymp[olymp['ypred'] == 1]

In [45]:
filtered_df['Name'].unique()

array([264, 308, 733, 451, 333, 603])

In [46]:
name_encoded_values = [264, 308, 733, 451, 333, 603]

# Filter the DataFrame to get the corresponding "Name" values
result = fxnames.loc[fxnames['Name_encoded'].isin(name_encoded_values), 'Name']


In [47]:
result.unique()

array(['harry hepworth', 'jake jarman', 'weide su', 'luke whitehouse',
       'jiaxing yang', 'rayderley zapata'], dtype=object)

In [48]:
fxolymp["Name"]=fxolymp["Name"].str.lower()

In [49]:
names_to_filter = ['harry hepworth', 'jake jarman', 'weide su', 'luke whitehouse',
       'jiaxing yang', 'rayderley zapata']

# Filter the DataFrame to get rows with the specified "Name" values
result = fxolymp[fxolymp['Name'].isin(names_to_filter)]


In [50]:
result['Country'].value_counts()

Country
GBR    208
CHN     32
ESP     27
ENG     15
Name: count, dtype: int64

In [51]:
usa = result[result['Country']=='USA']

In [52]:
usa['Name'].value_counts()

Series([], Name: count, dtype: int64)